# We will learn: 

- Difference between: 
    1) `reduceByKey()` and `reduce()`
    2) `reduceByKey()` and `groupBykey()`


# SparkSession

In [1]:
spark

## Create `SparkContext`

In [2]:
sc = spark.sparkContext
sc

<SparkContext master=yarn appName=PySparkShell>

# `reduceByKey()`

- Works on PAIR RDD 
    - ('hello', 1)
    - ('world', 1)
- Its a transformation 
- local aggregation takes place in all worker node (like combiner in MR)
    - As a result of this, shuffling would be less, as majority of the aggregation would be executed in every worker machine

In [13]:
data_set = 's3://fcc-spark-example/dataset/2023/orders.txt'

In [23]:
# Transformations 

rdd1 = sc.textFile(data_set)
rdd2 = rdd1.map(lambda line: (line.split(',')[-1], 1)) 


In [20]:
rdd2.take(5)

[('CLOSED', 1),
 ('PENDING_PAYMENT', 1),
 ('COMPLETE', 1),
 ('CLOSED', 1),
 ('COMPLETE', 1)]

In [24]:
rdd3 = rdd2.reduceByKey(lambda x, y: x+y)

In [25]:
rdd3.collect()

[('CLOSED', 7556),
 ('CANCELED', 1428),
 ('PENDING_PAYMENT', 15030),
 ('COMPLETE', 22899),
 ('PROCESSING', 8274),
 ('PAYMENT_REVIEW', 729),
 ('PENDING', 7609),
 ('ON_HOLD', 3798),
 ('SUSPECTED_FRAUD', 1558)]

# `reduce()`

- Its an action 
- Works on normal RDD
- Finally we get only single thing as an answer in the driver machine (thats why its an action)

In [6]:
some_data = list(range(10))
some_data

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [7]:
rdd1 = sc.parallelize(some_data)
rdd1.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [8]:
rdd1.reduce(lambda x, y: x + y)

45

In [11]:
rdd1.reduce(lambda x, y: max(x,y))

9

In [12]:
rdd1.reduce(lambda x, y: min(x,y))

0

# `reduceByKey()` vs `groupByKey()`

### Lets take a large dataset 
- `aws s3 ls s3://amazon-reviews-pds/tsv/` 
- Reference : [here](https://s3.amazonaws.com/amazon-reviews-pds/readme.html)
- We will use only books data :
    - amazon_reviews_us_Books_v1_00.tsv.gz 
    - amazon_reviews_us_Books_v1_01.tsv.gz
    - amazon_reviews_us_Books_v1_02.tsv.gz


In [3]:
dataset = 's3://amazon-reviews-pds/tsv/amazon_reviews_us_Books*'

In [4]:
rdd1 = sc.textFile(dataset)

In [47]:
rdd1.take(5)

['marketplace\tcustomer_id\treview_id\tproduct_id\tproduct_parent\tproduct_title\tproduct_category\tstar_rating\thelpful_votes\ttotal_votes\tvine\tverified_purchase\treview_headline\treview_body\treview_date',
 'US\t25933450\tRJOVP071AVAJO\t0439873800\t84656342\tThere Was an Old Lady Who Swallowed a Shell!\tBooks\t5\t0\t0\tN\tY\tFive Stars\tI love it and so does my students!\t2015-08-31',
 'US\t1801372\tR1ORGBETCDW3AI\t1623953553\t729938122\tI Saw a Friend\tBooks\t5\t0\t0\tN\tY\tPlease buy "I Saw a Friend"! Your children will be delighted!\tMy wife and I ordered 2 books and gave them as presents...one to a friend\'s daughter and the other to our grandson! Both children were so happy with the story, by author Katrina Streza, and they were overjoyed with the absolutely adorable artwork, by artist Michele Katz, throughout the book! We highly recommend &#34;I Saw a Friend&#34; to all your little ones!!!\t2015-08-31',
 'US\t5782091\tR7TNRFQAOUTX5\t142151981X\t678139048\tBlack Lagoon, Vol. 6

In [10]:
header = rdd1.first()

In [11]:
header

'marketplace\tcustomer_id\treview_id\tproduct_id\tproduct_parent\tproduct_title\tproduct_category\tstar_rating\thelpful_votes\ttotal_votes\tvine\tverified_purchase\treview_headline\treview_body\treview_date'

In [12]:
rdd2 = rdd1.filter(lambda x: x != header)

In [13]:
rdd3 = rdd2.map(lambda x: x.split('\t'))
#rdd3.take(1)

In [14]:
# Extracting only product ID (col #4)
rdd4 = rdd3.map(lambda x: (x[3], 1))
#rdd4.take(5)

In [15]:
result = rdd4.reduceByKey(lambda x, y : x + y)

In [16]:
result.collect()

[('142151981X', 4),
 ('1604600527', 4),
 ('0399170863', 254),
 ('0671728725', 3),
 ('1570913722', 9),
 ('161893158X', 12),
 ('B00ES29WSC', 13),
 ('0133083608', 48),
 ('1612785905', 141),
 ('191052428X', 10),
 ('0765362430', 307),
 ('089689293X', 225),
 ('1451624433', 85),
 ('0062270451', 72),
 ('1566845181', 10),
 ('0807070904', 4),
 ('B000KB9HD0', 3),
 ('0373677022', 1),
 ('1465047212', 15),
 ('1426767501', 27),
 ('0762741201', 14),
 ('0758287356', 17),
 ('0062091158', 2),
 ('0307098222', 73),
 ('1481979841', 28),
 ('1680283014', 4),
 ('1499505477', 57),
 ('0805092420', 4),
 ('161767737X', 1),
 ('0064408655', 107),
 ('1476727260', 19),
 ('0764210661', 114),
 ('1401945015', 137),
 ('1516979389', 1),
 ('193646053X', 3),
 ('0521552702', 1),
 ('0944508448', 3),
 ('0965793524', 6),
 ('8477117160', 1),
 ('0692429697', 2),
 ('0922658013', 11),
 ('1591164435', 9),
 ('0486438473', 11),
 ('1607103133', 393),
 ('0804126046', 10),
 ('1439173273', 167),
 ('158229285X', 3),
 ('0525421602', 36),
 ('

In [56]:
result.getNumPartitions()

3

# Summary 

<!-- - `map()` 

    - 100 ROWs =======> 1000 ROWs
    
    
- `reduceByKey()`

    - 100 ROWs =======> 10 ROWs (if there are 10 distinct Keys)
    
    
- `reduce()`

    - 100 ROWs =======> 1 ROW 
    
- `filter()` 

    - 100 ROWs =======> 0 <= No. of ROWs <= 100 (Depending on the filer ) -->
    


We learnt:
- Types of Transformation: 
    1) Narrow Transformation 
        - `map()`
        - `flatmap()`
        - `filter()`
    2) Wide Transformation 
        - `reduceByKey()`
        - `groupByKey()`
        
- We should always :
    - try to minimise `wide` transformation 
    - use `wide` transformation as late as possible 
    
- Spark History Server and what are :
    - Jobs
    - Stages 
    - Tasks 